# Part A
#### Question 1

Given:
$$\large \mu_1=(0.4, 0.8), \mu_2=(1.5, 2.7)$$
$$\large \Sigma = \begin{bmatrix}
1.5 & 0 \\
0 & 0.8
\end{bmatrix}$$
$$\large P(\omega_1) = 0.95, P(\omega_2) = 0.05 $$
Calculate
$$\large p(x|\omega_1), p(x|\omega_2)$$

#### Answer
We know that:
$$\large p\left(\boldsymbol{x} \mid \omega_i\right)=\frac{1}{(2 \pi)^{d / 2}|\boldsymbol{\Sigma}|^{1 / 2}} \exp \left[-\frac{1}{2}(\boldsymbol{x}-\boldsymbol{\mu})^t \boldsymbol{\Sigma}^{-1}(\boldsymbol{x}-\boldsymbol{\mu})\right]
$$
So we can convert the equation to python code in order to calculate $\large p(x|\omega_1), p(x|\omega_2)$


In [2]:
# Run this cell before anything else in order to import the necessary libraries and make the necessary variable declarations
import numpy as np
import scipy as sc

mu_1 = np.array([0.4, 0.8])
mu_2 = np.array([1.5, 2.7])

p_omega_1 = 0.95
p_omega_2 = 0.05

sigma = np.array([[1.5, 0], [0, .8]])
x = np.array([1, 0])

In [5]:
# Answer to question 1
def distribution_value(x, mu, sigma):
    d = mu.shape[0]
    p = np.zeros(x.shape[0])
    p = 1 / (pow((2 * np.pi), (d / 2)) * np.sqrt(np.linalg.det(sigma))) * np.exp(
        -0.5 * np.matmul(np.matmul(np.transpose(x - mu), np.linalg.inv(sigma)), (x - mu)))
    return p


print(distribution_value(x, mu_1, sigma))
print(distribution_value(x, mu_2, sigma))

0.08637665427744069
0.001403731574178327
